<a href="https://colab.research.google.com/github/mehedi832766/Real-time-Bengali-sign-language-detection-and-generate-words-from-detection-and-read-out-loud-/blob/main/YOLOv7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content

/content


In [ ]:
# Download YOLOv7 repository and install requirements
!git clone https://github.com/WongKinYiu/yolov7 
%cd yolov7
!pip install -r requirements.txt

# Your Custom Data

In [4]:
%cd /content

/content


In [5]:
!unzip "/content/drive/MyDrive/Thesis/data_yolo7.zip" 

Archive:  /content/drive/MyDrive/Thesis/data_yolo7.zip
  inflating: YOLOV&/classes.names    
   creating: YOLOV&/images/
   creating: YOLOV&/images/train/
  inflating: YOLOV&/images/train/@1A (1).jpg  
  inflating: YOLOV&/images/train/@1A (10).jpg  
  inflating: YOLOV&/images/train/@1A (11).jpg  
  inflating: YOLOV&/images/train/@1A (12).jpg  
  inflating: YOLOV&/images/train/@1A (13).jpg  
  inflating: YOLOV&/images/train/@1A (14).jpg  
  inflating: YOLOV&/images/train/@1A (15).jpg  
  inflating: YOLOV&/images/train/@1A (16).jpg  
  inflating: YOLOV&/images/train/@1A (17).jpg  
  inflating: YOLOV&/images/train/@1A (18).JPG  
  inflating: YOLOV&/images/train/@1A (19).JPG  
  inflating: YOLOV&/images/train/@1A (2).jpg  
  inflating: YOLOV&/images/train/@1A (20).JPG  
  inflating: YOLOV&/images/train/@1A (21).JPG  
  inflating: YOLOV&/images/train/@1A (22).JPG  
  inflating: YOLOV&/images/train/@1A (23).JPG  
  inflating: YOLOV&/images/train/@1A (24).JPG  
  inflating: YOLOV&/images/trai

# Prepare image path in txt file

In [6]:
import os

In [7]:
train_img_path = "/content/YOLOV&/images/train"
val_img_path = "/content/YOLOV&/images/val"

In [8]:
#Training images
with open('train.txt', "a+") as f:
  img_list = os.listdir(train_img_path)
  for img in img_list:
    f.write(os.path.join(train_img_path,img+'\n'))
  print("Done")

Done


In [9]:
# Validation Image
with open('val.txt', "a+") as f:
  img_list = os.listdir(val_img_path)
  for img in img_list:
    f.write(os.path.join(val_img_path,img+'\n'))
  print("Done")

Done


In [10]:
%cp /content/yolov7/data/coco.yaml /content/yolov7/data/custom.yaml

In [11]:
%cp /content/drive/MyDrive/Thesis/weights/best.pt /content/yolov7/best.pt 

In [12]:
%cp /content/yolov7/cfg/training/yolov7.yaml /content/yolov7/cfg/training/custom_yolov7.yaml

#Begin Training

In [13]:
cd yolov7

/content/yolov7


In [ ]:
!python /content/yolov7/train.py --batch 8 --cfg /content/yolov7/cfg/training/custom_yolov7.yaml --epochs 100 --data /content/yolov7/data/custom.yaml --weights 'best.pt' --device 0 


In [ ]:
# !python test.py --weights /content/yolov7/best.pt --data /content/yolov7/data/custom.yaml --verbose --save-txt --save-conf

In [ ]:
i=-1

In [ ]:
# Run 
!python detect.py --weights /content/yolov7/best.pt  --conf-thres .35 --source /content/drive/MyDrive/Thesis/YOLO/IMG_4019.MOV --save-txt --save-conf


i=i+1

In [ ]:
i

In [ ]:
path=os.listdir("/content/yolov7/runs/detect")
path = "/content/yolov7/runs/detect"+"/"+path[i]+"/"+"labels"

#helper function

Approach 2

In [ ]:
import glob
import pandas as pd

names= ['অ', 'আ', 'ই', 'উ', 'এ', 
        'ও','ক', 'খ', 'গ', 'ঘ', 'চ', 
        'ছ', 'জ', 'ঝ', 'ট', 'ঠ', 'ড', 
        'ঢ', 'ত', 'থ', 'ন', 'প', 'ফ', 
        'ব', 'ভ', 'ম', 'য়', 'র', 'ল', 
        'স', 'হ', 'ড়', 'UN', 'BI', 'joint', 
        'space', 'stop', 'দ', 'ধ']

def clpr(f):
  # print(f)
  #sum of confidence
  n1c=f.groupby('Class').agg(sum_conf=('conf', 'sum'))
  # print(f)
  #frequency
  n1f=f.groupby(['Class']).size().reset_index(name='freq')
  
  #merge
  f=(pd.merge(n1c, n1f, on='Class'))
  # print(f)
  #main result
  f["mul"]=f['freq']*f['sum_conf']
  #Class number
  # print(f)
  c=f['Class'][f['freq'].idxmax()]
  print(c)
  # print(names[c])
  return c






def txt_gen(p):
  read_files = glob.glob(p+"/*.txt")

  def myFunc(e):
    return len(e)

  read_files.sort(key=myFunc)


  with open(p+"/result.txt", "wb") as outfile:
      for f in read_files:
          with open(f, "rb") as infile:
              outfile.write(infile.read())





  # reading the given csv file
  # and creating dataframe
  account = pd.read_csv(p+"/result.txt",
            delimiter = ' ')
  account.columns = ['Class', 'x1', 'y1','x2','y2','conf']
  # store dataframe into csv file
  account.to_csv(p+'/yo4019.csv',
        index = None)


  file = pd.read_csv(p+"/yo4019.csv")

  f=file.shape[0]
  m=file.shape[0]//3
  m1=f-m

  f1=file.iloc[0:m]
  f2=file.iloc[m:m1]
  f3=file.iloc[m1:f]

 

  L=[]
  L.append(names[clpr(f1)])
  L.append(names[clpr(f2)])
  L.append(names[clpr(f3)])

  return L

In [ ]:
li=txt_gen(path)

In [ ]:
li

In [ ]:
str1=""
    # traverse in the string
for ele in li:
    str1 += ele

In [ ]:
str1

Approach 1

In [ ]:
def word_gen(p):
  read_files = glob.glob(p+"/*.txt")

  def myFunc(e):
    return len(e)

  read_files.sort(key=myFunc)

  l=[]

  for f in read_files:
      with open(f) as infile:
        for line in infile:
          pass
      li = list(line.split(" "))
      l.append(li[0])   

  df = pd.DataFrame(l,columns=['Class'])
  df.Class = df.Class.astype('int64')

  f=df.shape[0]
  m=df.shape[0]//3
  m1=f-m

  f1=df.iloc[0:m]
  f2=df.iloc[m:m1]
  f3=df.iloc[m1:f]

  names= ['অ', 'আ', 'ই', 'উ', 'এ', 
        'O','ক', 'খ', 'গ', 'ঘ', 'চ', 
        'ছ', 'জ', 'ঝ', 'ট', 'ঠ', 'ড', 
        'ঢ', 'ত', 'থ', 'ন', 'প', 'ফ', 
        'ব', 'ভ', 'ম', 'য়', 'র', 'ল', 
        'স', 'হ', 'ড়', 'UN', 'BI', 'joint', 
        'space', 'stop', 'দ', 'ধ']


  str1 = ""
  str1 += names[f1['Class'].value_counts().idxmax()]
  str1 += names[f2['Class'].value_counts().idxmax()]
  str1 += names[f3['Class'].value_counts().idxmax()]


  return str1

In [ ]:
li=word_gen(path)

In [ ]:
li

#read out loud

In [ ]:
pip install gTTS

In [ ]:
# Import the required module for text
# to speech conversion
from gtts import gTTS
from IPython.display import Audio, display
import os



In [ ]:
# def tG(s1):
#   names= ['অ', 'আ', 'ই', 'উ', 'এ', 
#         'O','ক', 'খ', 'গ', 'ঘ', 'চ', 
#         'ছ', 'জ', 'ঝ', 'ট', 'ঠ', 'ড', 
#         'ঢ', 'ত', 'থ', 'ন', 'প', 'ফ', 
#         'ব', 'ভ', 'ম', 'য়', 'র', 'ল', 
#         'স', 'হ', 'ড়', 'UN', 'BI', 'joint', 
#         'space', 'stop', 'দ', 'ধ']
#   print(s1)
#   list2=[]
#   for i in range(len(s1)):
#       if(len(list2)==0):
#         list2.append(s1[i])
#       elif((s1[i]!=list2[(len(list2))-1])):
#           list2.append(s1[i])

#   str1=""
      
#   for ele in list2:
#       str1 += names[ele]

#   return str1

In [ ]:
# def rOL(str1):
#   # Import the required module for text
#   # to speech conversion
#   from gtts import gTTS
#   from IPython.display import Audio, display
#   # This module is imported so that we can
#   # play the converted audio
#   import os

#   # The text that you want to convert to audio
#   # mytext = 'Welcome to geeksforgeeks!'

#   # Language in which you want to convert
#   language = 'bn'

#   # Passing the text and language to the engine,
#   # here we have marked slow=False. Which tells
#   # the module that the converted audio should
#   # have a high speed
#   myobj = gTTS(text=str1, lang=language, slow=False)
#   # time.sleep(5)
#   # # Saving the converted audio in a mp3 file named
#   # # welcome

#   myobj.save("welcom.wav")
#   # time.sleep(5)
#   sound_file = 'welcom.wav'
#   # print(sound_file)
#   # Audio(sound_file, autoplay=True)
#   # time.sleep(15)
#   return display(Audio(sound_file, autoplay=True))
 
  



# 7.1 Webcam Helper Functions

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import PIL
import io
import html
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes


# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
opt  = {
    
    "weights": "/content/yolov7/best.pt", # Path to weights file default weights are for nano model
    "yaml"   : "/content/yolov7/data/custom.yaml",
    "img-size": 640, # default image size
    "conf-thres": 0.35, # confidence threshold for inference.
    "iou-thres" : 0.5, # NMS IoU threshold for inference.
    "device" : '0',  # device to run our model i.e. 0 or 0,1,2,3 or cpu
    "classes" : ['stop','UN', 'BI']  # list of classes to filter or None

}

In [ ]:
import argparse
import time
from pathlib import Path
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel


def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

In [ ]:
def rOL(s1):
  names= ['অ', 'আ', 'ই', 'উ', 'এ', 
        'O','ক', 'খ', 'গ', 'ঘ', 'চ', 
        'ছ', 'জ', 'ঝ', 'ট', 'ঠ', 'ড', 
        'ঢ', 'ত', 'থ', 'ন', 'প', 'ফ', 
        'ব', 'ভ', 'ম', 'য়', 'র', 'ল', 
        'স', 'হ', 'ড়', 'UN', 'BI', 'joint', 
        'space', 'stop', 'দ', 'ধ']
  # print(s1)
  list2=[]
  for i in range(len(s1)):
      if(len(list2)==0):
        list2.append(s1[i])
      elif((s1[i]!=list2[(len(list2))-1])):
          list2.append(s1[i])
  str1=""    
  for ele in list2:
      str1 += names[ele]
  print(str1)
  # plot_one_box(xyxy, bbox_array, label=str1, color=colors[int(cls)], line_thickness=3)
  myobj = gTTS(text=str1, lang='bn', slow=True)
  myobj.save("/content/drive/MyDrive/Thesis/YOLO/welcome.wav")
  sound_file = '/content/drive/MyDrive/Thesis/YOLO/welcome.wav'

  return display(Audio(sound_file, autoplay=True))

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
s1=[]
with torch.no_grad():
  weights, imgsz = opt['weights'], (480,640)
  set_logging()
  device = select_device(opt['device'])
  half = device.type != 'cpu'
  model = attempt_load(weights, map_location=device)  # load FP32 model
  stride = int(model.stride.max())  # model stride

  if half:
    model.half()

  names = model.module.names if hasattr(model, 'module') else model.names
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
  if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz[0], imgsz[1]).to(device).type_as(next(model.parameters())))
  classes = None
  if opt['classes']:
    classes = []
    for class_name in opt['classes']:
      classes.append(opt['classes'].index(class_name))
  
  while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    
    img0 = js_to_image(js_reply["img"])
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    img = letterbox(img0, imgsz, stride=stride)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
      img = img.unsqueeze(0)

    # Inference
    t1 = time_synchronized()
    pred = model(img, augment= False)[0]
    
    # Apply NMS
    pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes= classes, agnostic= False)
    t2 = time_synchronized()
    for i, det in enumerate(pred):
      s = ''
      s += '%gx%g ' % img.shape[2:]  # print string
      gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
      if len(det):
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

        for c in det[:, -1].unique():
          n = (det[:, -1] == c).sum()  # detections per class
          s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
        
        for *xyxy, conf, cls in reversed(det):
          label = f'{names[int(cls)]} {conf:.2f}'
          if(names[int(cls)]!='space'):
            s1.append(int(cls))

          elif((names[int(cls)]=='space') and (len(s1)!=0)):
            rOL(s1)
            s1.clear()
          plot_one_box(xyxy, bbox_array, label=label, color=colors[int(cls)], line_thickness=3)
          
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)
    
    bbox = bbox_bytes
